In [1]:
# import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
# preprocessing
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [2]:
# import datasets 
train_labels = pd.read_csv("datasets/train_labels.csv", delimiter=",", header=0, index_col=0)
train_images = np.load("datasets/train_images.npy", encoding="latin1")
test_images = np.load("datasets/train_images.npy", encoding="latin1")

In [3]:
# store copies, may wish to retrieve original datasets
copy_train_labels = np.copy(train_labels)
copy_train_images = np.copy(train_images)
copy_test_images = np.copy(test_images)
 
# restore original datasets
def restore():
    train_labels = np.copy(copy_train_labels)
    train_images = np.copy(copy_train_images)
    test_images = np.copy(copy_test_images)
    return train_labels, train_images, test_images

In [4]:
# check shape
print(train_labels.shape)
print(train_images.shape)
print(test_images.shape)

(10000, 1)
(10000, 2)
(10000, 2)


In [5]:
# RUN FROM| FROM HERE IF DATASETS NEED TO BE RESTORED
train_labels, train_images, test_images = restore()

In [6]:
# reshape labels to 1d array, encode labels to integers
train_labels = np.array(train_labels).reshape(-1)
le = preprocessing.LabelEncoder()
le.fit(train_labels)
train_y = le.transform(train_labels)

In [7]:
# check labels
print(train_labels)
print(train_y)

['shovel' 'rifle' 'scorpion' ..., 'rollerskates' 'mouth' 'pencil']
[25 20 23 ..., 21  3 13]


In [ ]:
# reshape images
#train_images = train_images[:,1].reshape(-1, 100, 100)
train_x = []
for i in train_images[:,1]:
    train_x.append(i.reshape(100,100))
train_x = np.array(train_x, dtype=float) 

In [28]:
# check shape
print(train_x.shape)

(10000, 100, 100)
